In [61]:
import re
import json
import locale
import requests
import pandas as pd
from mplogger import Logger
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
from html2tei import parse_date

from html2tei import parse_date
from warcio.archiveiterator import ArchiveIterator
from webarticlecurator import WarcCachingDownloader

from origo_specific import get_meta_from_articles_spec as get_meta

## NOTES
- Ezzekkel a galéria cikkekkel mi lesz? https://www.origo.hu/auto/galeria/20220317-volkswagen-t7-multivan-etsi-teszt-2022-galeria.html

In [28]:
def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def response_warc_record_gen(warc_filename):
    archive_base = ArchiveIterator(open(warc_filename, 'rb'))
    for rec in archive_base:
        if rec.rec_type == 'response':
            article_url = rec.rec_headers.get_header('WARC-Target-URI')
            raw_html = rec.content_stream().read()
            yield article_url, raw_html

def extract_resp_record_data(resp):
    """Extract response date, WARC ID and raw HTML from a WARC response record"""
    warc_response_date = resp.rec_headers.get_header('WARC-Date')
    if '.' in warc_response_date:
        date_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    else:
        date_format = '%Y-%m-%dT%H:%M:%SZ'
    warc_response_datetime = datetime.strptime(warc_response_date, date_format)
    warc_id = resp.rec_headers.get_header('WARC-Record-ID')
    raw_html = resp.content_stream().read().decode(resp.rec_headers.get_header('WARC-X-Detected-Encoding'))

    return warc_response_datetime, warc_id, raw_html

In [5]:
logger = Logger()
warc_filename = '../../warcs_dir/origo-articles_new3.warc.gz'
w = WarcCachingDownloader(warc_filename, None, logger, just_cache=True, download_params={'stay_offline': True})

2022-03-24 12:57:34,120 INFO: Logging started
2022-03-24 12:57:34,121 INFO: Creating index for ../../warcs_dir/origo-articles_new3.warc.gz...
2022-03-24 13:01:42,063 INFO: Index successfully created.


In [45]:
link = list(w.url_index)[22222]

In [36]:
len(w.url_index
)

729885

In [75]:
# FROM WARC
_, _, resp = w.get_records(link4)
warc_response_datetime, warc_id, raw_html = extract_resp_record_data(resp)
soup = BeautifulSoup(raw_html, 'lxml')
get_meta(logger, link, soup)


2022-03-24 15:30:51,806 WARNING: https://www.origo.hu/sport/egyeni/20120223-klicsko-rendezne-a-szamlat-chisoraval.html: ARTICLE HEAD NOT FOUND!
2022-03-24 15:30:51,809 WARNING: https://www.origo.hu/sport/egyeni/20120223-klicsko-rendezne-a-szamlat-chisoraval.html: ARTICLE INFO NOT FOUND!
2022-03-24 15:30:51,811 WARNING: https://www.origo.hu/sport/egyeni/20120223-klicsko-rendezne-a-szamlat-chisoraval.html: ARTICLE BODY NOT FOUND!


In [31]:
l, raw_html = next(response_warc_record_gen(warc_filename))
bs = BeautifulSoup(raw_html, 'lxml')
get_meta(logger, l, bs)

Ezért veszélyes az LMBTQ-propaganda a gyerekekre
['Origo']
2022-03-17 14:57:00
Itthon
['LMBTQ', 'LMBTQ-propaganda', 'LMBTQ-lobbi']


In [74]:
link2 = 'https://www.origo.hu/sport/csapat/20220307-kezilabda-az-ihf-is-kizarta-az-oroszokat-es-a-feheroroszokat.html'
link3 = 'https://www.origo.hu/sport/futball/20220317-labdarugas-as-roma-nehezen-jutott-tovabb-a-konferencia-ligaban.html'
link4 = 'https://www.origo.hu/sport/galeria/20220317-a-szexi-luisinha-barosa-oliveira-mclaren-lando-norris-kepek-fotok-galeria.html'

In [43]:
article_head = soup.find('header', class_='article-head')

In [77]:
bs = soup

In [89]:
str(bs).find('2022031727.jpg')

13154

In [88]:
str(bs)[11300:11400]

'ght="1350" data-src="https://cdn.nwmgroups.hu/s/img/i/2203//2022031716.jpg" data-width="1080" src="/'

In [90]:
str(bs)[13100:13400]

'350" data-src="https://cdn.nwmgroups.hu/s/img/i/2203//2022031727.jpg" data-width="1080" src="//cdn.nwmgroups.hu/s/img/blank.png"/></div>\n<div class="caption-wrapper">\n<div class="container">\n<div class="row">\n<div class="col-sm-10 col-sm-offset-1 col-md-8 col-md-offset-2 nopadding">\n<div class="capt'